In [19]:
import torchvision
import  torch 
from torch import nn,from_numpy,cuda,optim,save,load
from torchvision.datasets import CIFAR100
from torchvision.transforms import ToTensor
from  torch import  nn,optim,cuda,from_numpy
from torch.utils import  data
from  torchvision import  datasets,transforms
import  torch.nn.functional as F
import matplotlib.pyplot as plt
import  numpy as np
import  cv2

In [20]:
device ='cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [21]:
class VGG16(torch.nn.Module):

    def __init__(self):
        super().__init__()
        
        self.block_1 = torch.nn.Sequential(
                torch.nn.Conv2d(in_channels=3,
                                out_channels=64,
                                kernel_size=(3, 3),
                                stride=(1, 1),
                                padding=1),
                torch.nn.ReLU(),
                torch.nn.Conv2d(in_channels=64,
                                out_channels=64,
                                kernel_size=(3, 3),
                                stride=(1, 1),
                                # (1(32-1)- 32 + 3)/2 = 1
                                padding=1),
                torch.nn.ReLU(),
                torch.nn.MaxPool2d(kernel_size=(2, 2),
                                   stride=(2, 2))
        )
        
        self.block_2 = torch.nn.Sequential(
                torch.nn.Conv2d(in_channels=64,
                                out_channels=128,
                                kernel_size=(3, 3),
                                stride=(1, 1),
                                padding=1),
                torch.nn.ReLU(),
                torch.nn.Conv2d(in_channels=128,
                                out_channels=128,
                                kernel_size=(3, 3),
                                stride=(1, 1),
                                padding=1),
                torch.nn.ReLU(),
                torch.nn.MaxPool2d(kernel_size=(2, 2),
                                   stride=(2, 2))
        )
        
        self.block_3 = torch.nn.Sequential(        
                torch.nn.Conv2d(in_channels=128,
                                out_channels=256,
                                kernel_size=(3, 3),
                                stride=(1, 1),
                                padding=1),
                torch.nn.ReLU(),
                torch.nn.Conv2d(in_channels=256,
                                out_channels=256,
                                kernel_size=(3, 3),
                                stride=(1, 1),
                                padding=1),
                torch.nn.ReLU(),        
                torch.nn.Conv2d(in_channels=256,
                                out_channels=256,
                                kernel_size=(3, 3),
                                stride=(1, 1),
                                padding=1),
                torch.nn.ReLU(),
                torch.nn.MaxPool2d(kernel_size=(2, 2),
                                   stride=(2, 2))
        )
        self.block_4 = torch.nn.Sequential(   
                torch.nn.Conv2d(in_channels=256,
                                out_channels=512,
                                kernel_size=(3, 3),
                                stride=(1, 1),
                                padding=1),
                torch.nn.ReLU(),        
                torch.nn.Conv2d(in_channels=512,
                                out_channels=512,
                                kernel_size=(3, 3),
                                stride=(1, 1),
                                padding=1),
                torch.nn.ReLU(),        
                torch.nn.Conv2d(in_channels=512,
                                out_channels=512,
                                kernel_size=(3, 3),
                                stride=(1, 1),
                                padding=1),
                torch.nn.ReLU(),            
                torch.nn.MaxPool2d(kernel_size=(2, 2),
                                   stride=(2, 2))
        )
        self.block_5 = torch.nn.Sequential(
                torch.nn.Conv2d(in_channels=512,
                                out_channels=512,
                                kernel_size=(3, 3),
                                stride=(1, 1),
                                padding=1),
                torch.nn.ReLU(),            
                torch.nn.Conv2d(in_channels=512,
                                out_channels=512,
                                kernel_size=(3, 3),
                                stride=(1, 1),
                                padding=1),
                torch.nn.ReLU(),            
                torch.nn.Conv2d(in_channels=512,
                                out_channels=512,
                                kernel_size=(3, 3),
                                stride=(1, 1),
                                padding=1),
                torch.nn.ReLU(),    
                torch.nn.MaxPool2d(kernel_size=(2, 2),
                                   stride=(2, 2))             
        )
        
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(512*1*1, 4096),
            torch.nn.ReLU(True),
            torch.nn.Dropout(p=0.5),
            torch.nn.Linear(4096, 4096),
            torch.nn.ReLU(True),
            torch.nn.Dropout(p=0.5),
            torch.nn.Linear(4096, 100),
        )
    def forward(self, x):

        x = self.block_1(x)
        x = self.block_2(x)
        x = self.block_3(x)
        x = self.block_4(x)
        x = self.block_5(x)
        x = x.view(x.size(0), -1) # flatten
        
        in_output = self.classifier(x)

        return in_output
    
model =VGG16()
model.load_state_dict(load('Cifar100_Vgg16.h5'))
model.eval()
turi = ['man','woman']

video = cv2.VideoCapture('Street.mp4')

while video.isOpened():
    boolean, kadr = video.read()
    face,confindence = cv.detect_face(kadr)
    
    for index,yuz in enumerate(face):
        (startX,startY,endX,endY) = yuz[0],yuz[1],yuz[2],yuz[3]
        
        yuz_np = np.copy(kadr[startY:endY,startX:endX])
        
        if yuz_np.shape[0] < 10 or yuz_np.shape[1] < 10:
            continue
            
        yuz_np = cv2.resize(yuz_np,(96,96))
        yuz_np = yuz_np.astype("float") / 255.0
        yuz_np = img_to_array(yuz_np)
        yuz_np = np.expand_dims(yuz_np,axis=0)
        
        bashorat = model.predict(yuz_np)[0]
        
        index = np.argmax(bashorat)
        label = turi[index]
        
        if label == 'man':
            color = (0,255,0)
        else:
            color = (0,0,255)
            
        label = f"{label} {np.around(bashorat[index]*100,2)}"
        
        if startY-10 > 10:
            Y=startY-10
        else:
            Y=startY+10
        
        cv2.rectangle(kadr,(startX,startY),(endX,endY),color,2)
        cv2.putText(kadr,label,(startX,Y),cv2.FONT_HERSHEY_SIMPLEX,0.7,color,2)
        
    cv2.imshow('Darsimiz AI',kadr)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video.release()
cv2.destroyAllWindows()

In [28]:
video =cv2.VideoCapture("1.MP4")
while video.isOpened():
    _,rasm =video.read()
    cv2.imshow("Video",rasm)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video.release()
cv2.destroyAllWindows()